In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats


In [3]:
df = pd.read_csv('../../data/sweep_0_3_data.csv')
df.drop(columns=["run_id", "run_name", "_runtime", "_step", "_timestamp"], inplace=True)

In [4]:
df.head()

,amd,rcm,padding,matrix_name,partial_gauss,rank,tile_size,max_mode_size,n,z,r2mode6
0,False,False,3,ex15,3,19818.0,3.0,109.0,6867.0,98640.0,6.586863e+20
1,False,False,3,ex15,3,8771.0,7.0,109.0,6867.0,98640.0,1.290201e+20
2,False,False,3,ex15,3,6821.0,9.0,109.0,6867.0,98640.0,7.802883e+19
3,False,False,3,ex15,3,2921.0,21.0,109.0,6867.0,98640.0,1.430942e+19
4,False,False,3,ex15,3,539.0,63.0,109.0,6867.0,98640.0,4.872328e+17


In [11]:
df["rank2"] = df["rank"]**2
df["max_mode_size6"] = df["max_mode_size"]**6

In [18]:
baseline_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]
padding_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0)]

In [19]:
baseline_df.head()

,amd,rcm,padding,matrix_name,partial_gauss,rank,tile_size,max_mode_size,n,z,r2mode6,rank2,max_mode_size6
274,False,False,0,ex15,0,19825.0,3.0,109.0,6867.0,98671.0,6.591517e+20,393030625.0,1.677100e+12
275,False,False,0,ex15,0,8125.0,7.0,109.0,6867.0,98671.0,1.107148e+20,66015625.0,1.677100e+12
276,False,False,0,ex15,0,6175.0,9.0,109.0,6867.0,98671.0,6.394888e+19,38130625.0,1.677100e+12
277,False,False,0,ex15,0,2275.0,21.0,109.0,6867.0,98671.0,8.680041e+18,5175625.0,1.677100e+12
278,False,False,0,ex15,0,325.0,63.0,109.0,6867.0,98671.0,1.771437e+17,105625.0,1.677100e+12


Show the trade-off between r and I, separate plot: r^2 vs I^6

In [17]:
fig = px.scatter(baseline_df, x="rank", y="max_mode_size", color="matrix_name", symbol="matrix_name", log_x=True, log_y=True, 
                 labels={
                     "rank": "Maximum Rank (r)",
                     "matrix_name": "Matrix Name",
                     "max_mode_size": "Maximum Mode Size (I)",
                 })
fig.update_layout(
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black') # Font color
)
fig.show()

fig.write_image("plots/max_mode_size_vs_max_rank.pdf")

In [16]:
fig = px.scatter(baseline_df, x="rank2", y="max_mode_size6", color="matrix_name", symbol="matrix_name", log_x=True, log_y=True, 
                 labels={
                     "rank2": "Maximum Rank Squared (r^2)",
                     "matrix_name": "Matrix Name",
                     "max_mode_size6": "I^6",
                 })
fig.update_layout(
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black') # Font color
)
fig.show()

fig.write_image("plots/max_mode_size6_vs_max_rank2.pdf")

In [34]:
padding_df_ex10 = padding_df[padding_df['matrix_name'] == "ex10hs"]
fig = px.scatter(padding_df_ex10, x="rank", y="max_mode_size", color="padding", log_x=True, log_y=True, 
                 labels={
                     "rank": "Maximum Rank Squared (r)",
                     "matrix_name": "Matrix Name",
                     "max_mode_size": "Maximum Mode Size (I)",
                 })
fig.update_layout(
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black'),  # Font color
    # coloraxis_colorbar=dict(yanchor="top", y=1, x=-0.2, ticks="outside", )
)
fig.show()

fig.write_image("plots/ex10_max_mode_size_vs_max_rank_vs_padding.pdf")